In [ ]:
1+1

In [ ]:
import pandas as pd
import numpy as np
import reweight_coh
import matplotlib.pyplot as plt
import matplotlib as mpl
import util
from numpy import linalg

import numiweight

In [ ]:
mpl.rc('font', size=14)

dosave = True
savedir = "./plots_2_28_24/coh-spectra/"

PIMASS = 0.139570
MUMASS = 0.1057

In [ ]:
import importlib

In [ ]:
filedir = "/icarus/data/users/gputnam/DMCP2023G/mc-F/"
mccohfile = filedir + "F-CohLike_nom_evt.df"

In [ ]:
mcdf = pd.read_hdf(mccohfile, key="mcnu")

In [ ]:
hdrdf = pd.read_hdf(mccohfile, key="hdr")

In [ ]:
mcpot = hdrdf.pot.sum()

GOAL_POT = 2.41e20

scale = GOAL_POT / mcpot
scale

In [ ]:
# include numiweight
ppfx_cv = numiweight.cv(mcdf.pdg, mcdf.E)
concrete_cv = numiweight.concrete_cv(mcdf.pdg, mcdf.E)

mcdf["numiweight"] = ppfx_cv * concrete_cv

In [ ]:
concrete_cv.mean()

In [ ]:
mcdf["scale"] = scale * mcdf.numiweight

In [ ]:
cccoh = (np.abs(mcdf.pdg) == 14) & (mcdf.iscc == 1) & (mcdf.genie_mode == 3)

In [ ]:
mu_reco_p = util.magdf(mcdf.p0.genp)
pi_reco_p = util.magdf(mcdf.p1.genp)*MUMASS/PIMASS

mu_reco_E = np.sqrt(mu_reco_p**2 + MUMASS**2)
pi_reco_E = np.sqrt(pi_reco_p**2 + MUMASS**2)

costh = util.dotdf(util.unitdf(mcdf.p0.genp), util.unitdf(mcdf.p1.genp))

muu = np.sqrt(2*MUMASS**2 + 2*(mu_reco_E*pi_reco_E - mu_reco_p*pi_reco_p*costh))

In [ ]:
beam2det = np.array([ [0.921035925, 0.022715103, 0.388814672], [0, 0.998297825, -0.058321970], [-0.389477631, 0.053716629, 0.919468161]])

beamorigin = np.array([4.503730e2, 80.153901e2, 795.112945e2])

MUON_MASS = 0.1057 
PION_MASS = 0.13457

BEAMDIR = beam2det.dot(beamorigin) / np.linalg.norm(beam2det.dot(beamorigin)) 

mcdf[("nu", "P", "x")] = mcdf.E*BEAMDIR[0]
mcdf[("nu", "P", "y")] = mcdf.E*BEAMDIR[1]
mcdf[("nu", "P", "z")] = mcdf.E*BEAMDIR[2]

nuclE = mcdf.E - mcdf.p0.genE - mcdf.p1.genE
nuclP = mcdf.nu.P - mcdf.p0.genp - mcdf.p1.genp
coht = np.abs(nuclE**2 - util.magdf(nuclP)**2)

In [ ]:
importlib.reload(reweight_coh)

cohweight = reweight_coh.cohweight(mcdf)

In [ ]:
cohweight[cccoh].cv

In [ ]:
thpi = np.arccos((mcdf.p1.genp.x*BEAMDIR[0] + mcdf.p1.genp.y*BEAMDIR[1] + mcdf.p1.genp.z*BEAMDIR[2]) / util.magdf(mcdf.p1.genp))*180/np.pi

In [ ]:
when = cccoh & util.SlcInFV(mcdf.position)

In [ ]:
NUNIV = 100

In [ ]:
bins = np.linspace(0, 6, 21)
v = mcdf.E

for i in range(NUNIV):
    label = "Syst. Variations" if i == 0 else None
    _ = plt.hist(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when], 
                 histtype="step", linewidth=0.5, color="lightgray", label=label)
    
_ = plt.hist(v[when], bins=bins, histtype="step", weights=mcdf.scale[when], linewidth=2, label="GENIE Nom.")
_ = plt.hist(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when], histtype="step", linewidth=2, label="Fit CV")
plt.ylabel("Fiducial Events / $2\\times 10^{20}$ POT")
plt.legend()

plt.xlabel("Neutrino Energy")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "nuE_univ.pdf")
    plt.savefig(savedir + "nuE_univ.svg")

In [ ]:
bins = np.linspace(0, 3, 13)
v = mcdf.p1.genE - PIMASS

for i in range(NUNIV):
    label = "Syst. Variations" if i == 0 else None
    _ = plt.hist(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when], 
                 histtype="step", linewidth=0.5, color="lightgray", label=label)
    
_ = plt.hist(v[when], bins=bins, histtype="step", weights=mcdf.scale[when], linewidth=2, label="GENIE Nom.")
_ = plt.hist(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when], histtype="step", linewidth=2, label="Fit CV")
plt.ylabel("Fiducial Events / $2\\times 10^{20}$ POT")
plt.legend()

plt.xlabel("Pion Kinetic Energy")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "piKE_univ.pdf")
    plt.savefig(savedir + "piKE_univ.svg")

In [ ]:
bins = np.linspace(0, 3, 13)
v = mcdf.p0.genE - MUMASS

for i in range(NUNIV):
    label = "Syst. Variations" if i == 0 else None
    _ = plt.hist(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when], 
                 histtype="step", linewidth=0.5, color="lightgray", label=label)
    
_ = plt.hist(v[when], bins=bins, histtype="step", weights=mcdf.scale[when], linewidth=2, label="GENIE Nom.")
_ = plt.hist(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when], histtype="step", linewidth=2, label="Fit CV")
plt.ylabel("Fiducial Events / $2\\times 10^{20}$ POT")
plt.legend()

plt.xlabel("Muon Kinetic Energy")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "muKE_univ.pdf")
    plt.savefig(savedir + "muKE_univ.svg")

In [ ]:
bins = np.linspace(0, 0.5, 11)
v = mcdf.Q2

for i in range(NUNIV):
    label = "Syst. Variations" if i == 0 else None
    _ = plt.hist(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when], 
                 histtype="step", linewidth=0.5, color="lightgray", label=label)
    
_ = plt.hist(v[when], bins=bins, histtype="step", weights=mcdf.scale[when], linewidth=2, label="GENIE Nom.")
_ = plt.hist(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when], histtype="step", linewidth=2, label="Fit CV")
plt.ylabel("Fiducial Events / $2\\times 10^{20}$ POT")
plt.legend()

plt.xlabel("$Q^2$")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "Q2_univ.pdf")
    plt.savefig(savedir + "Q2_univ.svg")

In [ ]:
bins = np.linspace(0, 1, 11) + 2*MUMASS
v = muu

for i in range(NUNIV):
    label = "Syst. Variations" if i == 0 else None
    _ = plt.hist(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when], 
                 histtype="step", linewidth=0.5, color="lightgray", label=label)
    
_ = plt.hist(v[when], bins=bins, histtype="step", weights=mcdf.scale[when], linewidth=2, label="GENIE Nom.")
_ = plt.hist(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when], histtype="step", linewidth=2, label="Fit CV")
plt.ylabel("Fiducial Events / $2\\times 10^{20}$ POT")
plt.legend()

plt.xlabel("$M_{\\mu\\mu}$")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "Muu_univ.pdf")
    plt.savefig(savedir + "Muu_univ.svg")

In [ ]:
bins = np.linspace(0, 3, 13)
v = mcdf.p1.genE - PIMASS

centers = (bins[1:] + bins[:-1]) / 2
Ns = []
for i in range(NUNIV):
    N,_ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when])
    Ns.append(N)
    
Nnom, _ = np.histogram(v[when], bins=bins, weights=mcdf.scale[when])
NCV, _ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when])

cov = np.zeros((NCV.size, NCV.size))

for N in Ns:
    cov += np.outer(NCV - N, NCV - N) / len(Ns)
    
corr = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))

cval, cvec = linalg.eig(cov)
cvec0 = cvec[:, 0]

ax1 = plt.gca()
ax1.set_zorder(4)
ax2 = ax1.twinx()
ax2.set_zorder(3)
_,_,s0 = ax2.hist(centers, bins=bins, weights=100*np.sqrt(np.diag(cov))/NCV, color="lightgray", label="Total")
_,_,s1 = ax2.hist(centers, bins=bins, weights=100*np.abs(cvec0)*np.sqrt(cval[0])/NCV, color="gray", label="1st P.C.")
ax2.set_ylim([0, 100])
ax2.set_ylabel("% Uncertainty", rotation=270)

_,_,h0 = ax1.hist(centers, bins=bins, weights=Nnom, label="GENIE Nom.", histtype="step", linewidth=2)
_,_,h1 = ax1.hist(centers, bins=bins, weights=NCV, label="Fit CV", histtype="step", linewidth=2)
ax1.patch.set_visible(False)

l1 = plt.legend(handles=[h0[0], h1[0]], title="Events", loc="upper right")
plt.legend(handles=[s0[0], s1[0]], title="Syst. Uncertainty", loc="center right")
plt.gca().add_artist(l1)
ax1.set_ylabel("Fiducial Events / $2\\times 10^{20}$ POT")

ax1.set_xlabel("Pion Kinetic Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "piKE.pdf")
    plt.savefig(savedir + "piKE.svg")

In [ ]:
bins = np.linspace(0, 5, 16)
v = mcdf.p0.genE - MUMASS

centers = (bins[1:] + bins[:-1]) / 2
Ns = []
for i in range(NUNIV):
    N,_ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when])
    Ns.append(N)
    
Nnom, _ = np.histogram(v[when], bins=bins, weights=mcdf.scale[when])
NCV, _ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when])

cov = np.zeros((NCV.size, NCV.size))

for N in Ns:
    cov += np.outer(NCV - N, NCV - N) / len(Ns)
    
corr = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))

cval, cvec = linalg.eig(cov)
cvec0 = cvec[:, 0]

ax1 = plt.gca()
ax1.set_zorder(4)
ax2 = ax1.twinx()
ax2.set_zorder(3)
_,_,s0 = ax2.hist(centers, bins=bins, weights=100*np.sqrt(np.diag(cov))/NCV, color="lightgray", label="Total")
_,_,s1 = ax2.hist(centers, bins=bins, weights=100*np.abs(cvec0)*np.sqrt(cval[0])/NCV, color="gray", label="1st P.C.")
ax2.set_ylim([0, 100])
ax2.set_ylabel("% Uncertainty", rotation=270)

_,_,h0 = ax1.hist(centers, bins=bins, weights=Nnom, label="GENIE Nom.", histtype="step", linewidth=2)
_,_,h1 = ax1.hist(centers, bins=bins, weights=NCV, label="Fit CV", histtype="step", linewidth=2)
ax1.patch.set_visible(False)

l1 = plt.legend(handles=[h0[0], h1[0]], title="Events", loc="upper right")
plt.legend(handles=[s0[0], s1[0]], title="Syst. Uncertainty", loc="center right", bbox_to_anchor=(1, 0.59))
plt.gca().add_artist(l1)
ax1.set_ylabel("Fiducial Events / $2\\times 10^{20}$ POT")

ax1.set_xlabel("Muon Kinetic Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "muKE.pdf")
    plt.savefig(savedir + "muKE.svg")

In [ ]:
bins = np.linspace(0, 6, 13)
v = mcdf.E

centers = (bins[1:] + bins[:-1]) / 2
Ns = []
for i in range(NUNIV):
    N,_ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when])
    Ns.append(N)
    
Nnom, _ = np.histogram(v[when], bins=bins, weights=mcdf.scale[when])
NCV, _ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when])

cov = np.zeros((NCV.size, NCV.size))

for N in Ns:
    cov += np.outer(NCV - N, NCV - N) / len(Ns)
    
corr = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))

cval, cvec = linalg.eig(cov)
cvec0 = cvec[:, 0]

ax1 = plt.gca()
ax1.set_zorder(4)
ax2 = ax1.twinx()
ax2.set_zorder(3)
_,_,s0 = ax2.hist(centers, bins=bins, weights=100*np.sqrt(np.diag(cov))/NCV, color="lightgray", label="Total")
_,_,s1 = ax2.hist(centers, bins=bins, weights=100*np.abs(cvec0)*np.sqrt(cval[0])/NCV, color="gray", label="1st P.C.")
ax2.set_ylim([0, 100])
ax2.set_ylabel("% Uncertainty", rotation=270)

_,_,h0 = ax1.hist(centers, bins=bins, weights=Nnom, label="GENIE Nom.", histtype="step", linewidth=2)
_,_,h1 = ax1.hist(centers, bins=bins, weights=NCV, label="Fit CV", histtype="step", linewidth=2)
ax1.patch.set_visible(False)

l1 = plt.legend(handles=[h0[0], h1[0]], title="Events", loc="upper right")
plt.legend(handles=[s0[0], s1[0]], title="Syst. Uncertainty", loc="center right", bbox_to_anchor=(1, 0.59))
plt.gca().add_artist(l1)
ax1.set_ylabel("Fiducial Events / $2\\times 10^{20}$ POT")

ax1.set_xlabel("Neutrino Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "nuE.pdf")
    plt.savefig(savedir + "nuE.svg")

In [ ]:
bins = np.linspace(0, 0.5, 11)
v = mcdf.Q2

centers = (bins[1:] + bins[:-1]) / 2
Ns = []
for i in range(NUNIV):
    N,_ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when])
    Ns.append(N)
    
Nnom, _ = np.histogram(v[when], bins=bins, weights=mcdf.scale[when])
NCV, _ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when])

cov = np.zeros((NCV.size, NCV.size))

for N in Ns:
    cov += np.outer(NCV - N, NCV - N) / len(Ns)
    
corr = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))

cval, cvec = linalg.eig(cov)
cvec0 = cvec[:, 0]

ax1 = plt.gca()
ax1.set_zorder(4)
ax2 = ax1.twinx()
ax2.set_zorder(3)
_,_,s0 = ax2.hist(centers, bins=bins, weights=100*np.sqrt(np.diag(cov))/NCV, color="lightgray", label="Total")
_,_,s1 = ax2.hist(centers, bins=bins, weights=100*np.abs(cvec0)*np.sqrt(cval[0])/NCV, color="gray", label="1st P.C.")
ax2.set_ylim([0, 100])
ax2.set_ylabel("% Uncertainty", rotation=270)

_,_,h0 = ax1.hist(centers, bins=bins, weights=Nnom, label="GENIE Nom.", histtype="step", linewidth=2)
_,_,h1 = ax1.hist(centers, bins=bins, weights=NCV, label="Fit CV", histtype="step", linewidth=2)
ax1.patch.set_visible(False)

l1 = plt.legend(handles=[h0[0], h1[0]], title="Events", loc="upper right")
plt.legend(handles=[s0[0], s1[0]], title="Syst. Uncertainty", loc="center right", bbox_to_anchor=(1, 0.59))
plt.gca().add_artist(l1)
ax1.set_ylabel("Fiducial Events / $2\\times 10^{20}$ POT")

ax1.set_xlabel("$Q^2$")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "Q2.pdf")
    plt.savefig(savedir + "Q2.svg")

In [ ]:
bins = np.linspace(0, 1, 11) + 2*MUMASS
v = muu

centers = (bins[1:] + bins[:-1]) / 2
Ns = []
for i in range(NUNIV):
    N,_ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when])
    Ns.append(N)
    
Nnom, _ = np.histogram(v[when], bins=bins, weights=mcdf.scale[when])
NCV, _ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when])

cov = np.zeros((NCV.size, NCV.size))

for N in Ns:
    cov += np.outer(NCV - N, NCV - N) / len(Ns)
    
corr = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))

cval, cvec = linalg.eig(cov)
cvec0 = cvec[:, 0]

ax1 = plt.gca()
ax1.set_zorder(4)
ax2 = ax1.twinx()
ax2.set_zorder(3)
_,_,s0 = ax2.hist(centers, bins=bins, weights=100*np.sqrt(np.diag(cov))/NCV, color="lightgray", label="Total")
_,_,s1 = ax2.hist(centers, bins=bins, weights=100*np.abs(cvec0)*np.sqrt(cval[0])/NCV, color="gray", label="1st P.C.")
ax2.set_ylim([0, 100])
ax2.set_ylabel("% Uncertainty", rotation=270)

_,_,h0 = ax1.hist(centers, bins=bins, weights=Nnom, label="GENIE Nom.", histtype="step", linewidth=2)
_,_,h1 = ax1.hist(centers, bins=bins, weights=NCV, label="Fit CV", histtype="step", linewidth=2)
ax1.patch.set_visible(False)

l1 = plt.legend(handles=[h0[0], h1[0]], title="Events", loc="upper right")
plt.legend(handles=[s0[0], s1[0]], title="Syst. Uncertainty", loc="center right", bbox_to_anchor=(1, 0.59))
plt.gca().add_artist(l1)
ax1.set_ylabel("Fiducial Events / $2\\times 10^{20}$ POT")

ax1.set_xlabel("$M_{\\mu\\mu}$ [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "muu.pdf")
    plt.savefig(savedir + "muu.svg")

In [ ]:
bins = np.linspace(0, 0.1, 11)
v = coht

centers = (bins[1:] + bins[:-1]) / 2
Ns = []
for i in range(NUNIV):
    N,_ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when])
    Ns.append(N)
    
Nnom, _ = np.histogram(v[when], bins=bins, weights=mcdf.scale[when])
NCV, _ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when])

cov = np.zeros((NCV.size, NCV.size))

for N in Ns:
    cov += np.outer(NCV - N, NCV - N) / len(Ns)
    
corr = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))

cval, cvec = linalg.eig(cov)
cvec0 = cvec[:, 0]

ax1 = plt.gca()
ax1.set_zorder(4)
ax2 = ax1.twinx()
ax2.set_zorder(3)
_,_,s0 = ax2.hist(centers, bins=bins, weights=100*np.sqrt(np.diag(cov))/NCV, color="lightgray", label="Total")
_,_,s1 = ax2.hist(centers, bins=bins, weights=100*np.abs(cvec0)*np.sqrt(cval[0])/NCV, color="gray", label="1st P.C.")
ax2.set_ylim([0, 100])
ax2.set_ylabel("% Uncertainty", rotation=270)

_,_,h0 = ax1.hist(centers, bins=bins, weights=Nnom, label="GENIE Nom.", histtype="step", linewidth=2)
_,_,h1 = ax1.hist(centers, bins=bins, weights=NCV, label="Fit CV", histtype="step", linewidth=2)
ax1.patch.set_visible(False)

l1 = plt.legend(handles=[h0[0], h1[0]], title="Events", loc="upper center", bbox_to_anchor=(0.375, 1))
plt.legend(handles=[s0[0], s1[0]], title="Syst. Uncertainty", loc="upper right")
plt.gca().add_artist(l1)
ax1.set_ylabel("Fiducial Events / $2\\times 10^{20}$ POT")

ax1.set_xlabel("|t| [GeV$^2$]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "coht.pdf")
    plt.savefig(savedir + "coht.svg")

In [ ]:
bins = np.linspace(0, 60, 13)
v = thpi

centers = (bins[1:] + bins[:-1]) / 2
Ns = []
for i in range(NUNIV):
    N,_ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*cohweight["univ%i" % i][when]*mcdf.scale[when])
    Ns.append(N)
    
Nnom, _ = np.histogram(v[when], bins=bins, weights=mcdf.scale[when])
NCV, _ = np.histogram(v[when], bins=bins, weights=cohweight.cv[when]*mcdf.scale[when])

cov = np.zeros((NCV.size, NCV.size))

for N in Ns:
    cov += np.outer(NCV - N, NCV - N) / len(Ns)
    
corr = cov / np.outer(np.sqrt(np.diag(cov)), np.sqrt(np.diag(cov)))

cval, cvec = linalg.eig(cov)
cvec0 = cvec[:, 0]

ax1 = plt.gca()
ax1.set_zorder(4)
ax2 = ax1.twinx()
ax2.set_zorder(3)
_,_,s0 = ax2.hist(centers, bins=bins, weights=100*np.sqrt(np.diag(cov))/NCV, color="lightgray", label="Total")
_,_,s1 = ax2.hist(centers, bins=bins, weights=100*np.abs(cvec0)*np.sqrt(cval[0])/NCV, color="gray", label="1st P.C.")
ax2.set_ylim([0, 100])
ax2.set_ylabel("% Uncertainty", rotation=270)

_,_,h0 = ax1.hist(centers, bins=bins, weights=Nnom, label="GENIE Nom.", histtype="step", linewidth=2)
_,_,h1 = ax1.hist(centers, bins=bins, weights=NCV, label="Fit CV", histtype="step", linewidth=2)
ax1.patch.set_visible(False)

l1 = plt.legend(handles=[h0[0], h1[0]], title="Events", loc="upper right")
plt.legend(handles=[s0[0], s1[0]], title="Syst. Uncertainty", loc="center right")
plt.gca().add_artist(l1)
ax1.set_ylabel("Fiducial Events / $2\\times 10^{20}$ POT")

ax1.set_xlabel("$\\theta_\\pi$ [$^\\circ$]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "thpi.pdf")
    plt.savefig(savedir + "thpi.svg")

In [ ]:
vx = muu
binx = np.linspace(0, 1, 11) + 2*MUMASS

vy = mcdf.p0.genE - MUMASS
biny = np.linspace(0, 3, 13)

_ = plt.hist2d(vx[when], vy[when], bins=[binx, biny], weights=cohweight.cv[when]*mcdf.scale[when])
cbar = plt.colorbar()
cbar.set_label("Fiducial Events / $2\\times 10^{20}$ POT", rotation=270, labelpad=15)

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("Muon Kinetic Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "muKE_muu_2D.pdf")
    plt.savefig(savedir + "muKE_muu_2D.svg")

In [ ]:
maxweight = cohweight.cv[when].max()

reweight = np.random.rand(when.sum()) < cohweight.cv[when] / maxweight

In [ ]:
vx = muu
binx = np.linspace(0, 1, 11) + 2*MUMASS

vy = mcdf.p0.genE - MUMASS
biny = np.linspace(0, 3, 13)

_ = plt.scatter(vx[when][reweight], vy[when][reweight], s=0.5)

plt.xlim([2*MUMASS, 1+2*MUMASS])
plt.ylim([0,3])

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("Muon Kinetic Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "muKE_muu_scatter.pdf")
    plt.savefig(savedir + "muKE_muu_scatter.svg")

In [ ]:
vx = muu
binx = np.linspace(0, 1, 11) + 2*MUMASS

vy = mcdf.p1.genE - PIMASS
biny = np.linspace(0, 3, 13)

_ = plt.hist2d(vx[when], vy[when], bins=[binx, biny], weights=cohweight.cv[when]*mcdf.scale[when])
cbar = plt.colorbar()
cbar.set_label("Fiducial Events / $2\\times 10^{20}$ POT", rotation=270, labelpad=15)

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("Pion Kinetic Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "piKE_muu_2D.pdf")
    plt.savefig(savedir + "piKE_muu_2D.svg")

In [ ]:
vx = muu
binx = np.linspace(0, 1, 11) + 2*MUMASS

vy = mcdf.p1.genE - PIMASS
biny = np.linspace(0, 3, 13)

_ = plt.scatter(vx[when][reweight], vy[when][reweight], s=0.5)

plt.xlim([2*MUMASS, 1+2*MUMASS])
plt.ylim([0,3])

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("Pion Kinetic Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "piKE_muu_scatter.pdf")
    plt.savefig(savedir + "piKE_muu_scatter.svg")

In [ ]:
vx = muu
binx = np.linspace(0, 1, 11) + 2*MUMASS

vy = (mcdf.E - mcdf.p0.genE) / mcdf.E
biny = np.linspace(0, 1, 11)

_ = plt.hist2d(vx[when], vy[when], bins=[binx, biny], weights=cohweight.cv[when]*mcdf.scale[when])
cbar = plt.colorbar()
cbar.set_label("Fiducial Events / $2\\times 10^{20}$ POT", rotation=270, labelpad=15)

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("y")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "y_muu_2D.pdf")
    plt.savefig(savedir + "y_muu_2D.svg")

In [ ]:
vx = muu
binx = np.linspace(0, 1, 11) + 2*MUMASS

vy = (mcdf.E - mcdf.p0.genE) / mcdf.E
biny = np.linspace(0, 1, 11)

_ = plt.scatter(vx[when][reweight], vy[when][reweight], s=0.5)
plt.xlim([2*MUMASS, 1+2*MUMASS])

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("y")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "y_muu_scatter.pdf")
    plt.savefig(savedir + "y_muu_scatter.svg")

In [ ]:
vx = muu
binx = np.linspace(0, 1, 11) + 2*MUMASS

vy = mcdf.Q2
biny = np.linspace(0, 0.5, 11)

_ = plt.hist2d(vx[when], vy[when], bins=[binx, biny], weights=cohweight.cv[when]*mcdf.scale[when])
cbar = plt.colorbar()
cbar.set_label("Fiducial Events / $2\\times 10^{20}$ POT", rotation=270, labelpad=15)

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("$Q^2$ [GeV$^2$]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "Q2_muu_2D.pdf")
    plt.savefig(savedir + "Q2_muu_2D.svg")

In [ ]:
vx = muu
binx = np.linspace(0, 1, 11) + 2*MUMASS

vy = coht
biny = np.linspace(0, 0.1, 11)

_ = plt.hist2d(vx[when], vy[when], bins=[binx, biny], weights=cohweight.cv[when]*mcdf.scale[when])
cbar = plt.colorbar()
cbar.set_label("Fiducial Events / $2\\times 10^{20}$ POT", rotation=270, labelpad=15)

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("$|t|$ [GeV$^2$]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "coht_muu_2D.pdf")
    plt.savefig(savedir + "coht_muu_2D.svg")

In [ ]:
vx = muu
vy = coht

_ = plt.scatter(vx[when][reweight], vy[when][reweight], s=0.5)
plt.xlim([2*MUMASS, 1+2*MUMASS])
plt.ylim([0, 0.1])

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("|t|")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "coht_muu_scatter.pdf")
    plt.savefig(savedir + "coht_muu_scatter.svg")

In [ ]:
vx = muu
binx = np.linspace(0, 1, 11) + 2*MUMASS

vy = thpi
biny = np.linspace(0, 60, 13)

_ = plt.hist2d(vx[when], vy[when], bins=[binx, biny], weights=cohweight.cv[when]*mcdf.scale[when])
cbar = plt.colorbar()
cbar.set_label("Fiducial Events / $2\\times 10^{20}$ POT", rotation=270, labelpad=15)

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("$\\theta_{\\pi}$ [$^\\circ$]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "thpi_muu_2D.pdf")
    plt.savefig(savedir + "thpi_muu_2D.svg")

In [ ]:
vx = muu
vy = thpi

_ = plt.scatter(vx[when][reweight], vy[when][reweight], s=0.5)
plt.xlim([2*MUMASS, 1+2*MUMASS])
plt.ylim([0, 60])

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("$\\theta_{\\pi}$ [$^\\circ$]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "thpi_muu_scatter.pdf")
    plt.savefig(savedir + "thpi_muu_scatter.svg")

In [ ]:
vx = muu
binx = np.linspace(0, 1, 11) + 2*MUMASS

vy = mcdf.E
biny = np.linspace(0, 5, 11)

_ = plt.hist2d(vx[when], vy[when], bins=[binx, biny], weights=cohweight.cv[when]*mcdf.scale[when])
cbar = plt.colorbar()
cbar.set_label("Fiducial Events / $2\\times 10^{20}$ POT", rotation=270, labelpad=15)

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("Neutrino Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "nuE_muu_2D.pdf")
    plt.savefig(savedir + "nuE_muu_2D.svg")

In [ ]:
vx = muu
vy = mcdf.E

_ = plt.scatter(vx[when][reweight], vy[when][reweight], s=0.5)
plt.xlim([2*MUMASS, 1+2*MUMASS])
plt.ylim([0, 6])

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.ylabel("Neutrino Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "nuE_muu_scatter.pdf")
    plt.savefig(savedir + "nuE_muu_scatter.svg")

In [ ]:
vx = thpi
binx = np.linspace(0, 60, 13)

vy = coht
biny = np.linspace(0, 0.1, 11)

_ = plt.hist2d(vx[when], vy[when], bins=[binx, biny], weights=cohweight.cv[when]*mcdf.scale[when])
cbar = plt.colorbar()
cbar.set_label("Fiducial Events / $2\\times 10^{20}$ POT", rotation=270, labelpad=15)

plt.xlabel("$M_{\\mu\\mu}$ [GeV]")
plt.xlabel("$\\theta_\\pi$ [$^\\circ$]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "t_thpi_2D.pdf")
    plt.savefig(savedir + "t_thpi_2D.svg")

In [ ]:
vx = thpi
vy = coht

_ = plt.scatter(vx[when][reweight], vy[when][reweight], s=0.5)
plt.xlim([0, 60])
plt.ylim([0, 0.1])

plt.xlabel("$\\theta_\\pi$ [$^\\circ$]")
plt.ylabel("|$t$| [GeV$^2$]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "t_thpi_scatter.pdf")
    plt.savefig(savedir + "t_thpi_scatter.svg")

In [ ]:
vx = thpi
binx = np.linspace(0, 60, 13)

vy = mcdf.Q2
biny = np.linspace(0, 0.2, 11)

_ = plt.hist2d(vx[when], vy[when], bins=[binx, biny], weights=cohweight.cv[when]*mcdf.scale[when])
cbar = plt.colorbar()
cbar.set_label("Fiducial Events / $2\\times 10^{20}$ POT", rotation=270, labelpad=15)

plt.xlabel("$\\theta_\\pi$ [$^\\circ$]")
plt.ylabel("$Q^2$ [GeV$^2$]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "Q2_thpi_2D.pdf")
    plt.savefig(savedir + "Q2_thpi_2D.svg")

In [ ]:
vx = thpi
vy = mcdf.Q2

_ = plt.scatter(vx[when][reweight], vy[when][reweight], s=0.5)
plt.xlim([0, 60])
plt.ylim([0, 0.1])

plt.xlabel("$\\theta_\\pi$ [$^\\circ$]")
plt.ylabel("$Q^2$ [GeV$^2$]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "Q2_thpi_scatter.pdf")
    plt.savefig(savedir + "Q2_thpi_scatter.svg")